In [214]:
import datetime
import multiprocessing
import concurrent
import subprocess
import uuid
from iris.fileformats.pp import load_pairs_from_fields
import numpy as np
import logging
import warnings
import os, sys
import scipy.ndimage as ndimage
from skimage import measure
#from tqdm import tqdm
import pandas as pd
import iris
import iris.quickplot as qplt
import iris.plot as iplt
import warnings
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Polygon, MultiPoint, Point
warnings.filterwarnings('ignore')
%matplotlib tk

In [77]:
def print_progress_bar(iteration, total):
    percentage = 100 * iteration / total
    progress_bar = f"     Progress: [{percentage:.2f}%]"
    print(progress_bar, end="\r")
    
def generate_mask(cube_data, threshold, threshold_method):
    '''
    Generate masks based on threshold and threshold method
    e.g. mask precip >= 10 uses threshold=10, threshold_method='geq'
    e.g. mask OLR <= 240 uses threshold=240, threshold_method='leq'
    :param cube_data:
    :type cube_data:
    :param threshold:
    :type threshold:
    :param threshold_method: 'leq' for <= or 'geq' for >=
    :type threshold_method:
    :return:
    :rtype:
    '''
    if threshold_method == 'geq':
        mask = cube_data >= threshold
    elif threshold_method == 'leq':
        mask = cube_data <= threshold
    return mask
    
def grid_features(cube, thresholds=None, time_index=0, member=0, threshold_method='geq'):
    '''
    2D cube tracking for thresholds
    :param cube: Lat-lon cube
    :type cube:
    :param thresholds:
    :type thresholds:
    :param time_index:
    :type time_index:
    :param threshold_method:
    :type threshold_method:
    :return: DataFrame of identified objects and their properties
    :rtype: Pandas DataFrame
    '''
    assert thresholds is not None, "Threshold values not found."

    # indices = []
    time_indices = []
    mem_indices = []
    cube_dates = []
    object_coords = []
    object_labels = []
    threshold_values = []
    areas = []
    perimeters = []
    eccs = []
    orients = []
    centroids = []
    mean_values = []
    std_values = []
    max_values = []
    min_values = []
    ngrid_points = []
    forecast_period = []
    forecast_reference_time = []
    data_values = []
    surface_type = []
    index = time_index
    if cube.ndim == 2:
        ny, nx = cube.shape
        lons, lats = cube.coord('longitude').points, cube.coord('latitude').points

        # Cube date
        if cube.coords('time'):
            c_date = cube.coord('time').units.num2date(cube.coord('time').points)[0]
            cube_date = datetime.datetime(c_date.year, c_date.month, c_date.day)

        if cube.coords('forecast_reference_time'):
            frt = cube.coord('forecast_reference_time').units.num2date(
                cube.coord('forecast_reference_time').points)[0]
            forecast_rt = datetime.datetime(frt.year, frt.month, frt.day)
        else:
            forecast_rt = np.nan

        if cube.coords('forecast_period'):
            forecast_p = cube.coord('forecast_period').points[0]
        else:
            forecast_p = np.nan

        for threshold in thresholds:
            # print('Thresholding %s' %threshold)
            cube_data = cube.data.copy()
            mask = generate_mask(cube_data, threshold, threshold_method)

            # Label each feature in the mask
            labeled_array, num_features = ndimage.measurements.label(mask)
            # print('%s features labelled.' % num_features)
            # labelled_array is a mask hence != operator below
            for feature_num in range(1, num_features):
                #print_progress_bar(feature_num + 1, num_features)

                # threshold
                threshold_values.append(threshold)
                object_labels.append(f'{index}_{member}_{threshold}_{feature_num}')
                loc = labeled_array != feature_num
                data_object = np.ma.masked_array(cube_data, loc)

                ###### Skimage needs the mask reversed
                lab_image = measure.label(labeled_array == feature_num)
                region = measure.regionprops(lab_image, np.ma.masked_array(cube_data, ~loc))

                # perimeter, eccentricity, orientation
                areas.append([p.area for p in region][0])
                perimeters.append([p.perimeter for p in region][0])
                eccs.append([p.eccentricity for p in region][0])
                orients.append([p.orientation for p in region][0])
                # print(eccs)
                ###############

                data_values.append(data_object.compressed())
                mean_values.append(np.ma.mean(data_object))
                std_values.append(np.ma.std(data_object))
                max_values.append(np.ma.max(data_object))
                min_values.append(np.ma.min(data_object))

                try:
                    y, x = ndimage.measurements.center_of_mass(data_object)
                    centroids.append((lons[round(x)], lats[round(y)]))
                except:
                    centroids.append((np.nan, np.nan))

                object_inds = np.where(loc == False)
                object_lats = [lats[i] for i in object_inds[0]]
                object_lons = [lons[i] for i in object_inds[1]]

                object_coords.append([(x, y) for x, y in zip(object_lons, object_lats)])

                # surface type
                # This slows the computation down significantly
                # surface_type.append(check_land_or_ocean(object_lons, object_lats))

                ngrid_points.append(len(object_lats))

                cube_dates.append(cube_date)
                forecast_period.append(forecast_p)
                forecast_reference_time.append(forecast_rt)
                # indices.append(index)
                time_indices.append(index)
                mem_indices.append(member)

        index += 1
    features = {'TimeInds': time_indices, 'Date': cube_dates,
                'Forecast_period': forecast_period, 'Forecast_reference_time': forecast_reference_time,
                'Member': mem_indices,
                'Threshold': threshold_values, 'ObjectLabel': object_labels, 'Area': areas,
                'GridPoints': ngrid_points,
                'Mean': mean_values, 'Std': std_values,
                'Max': max_values, 'Min': min_values,
                'Centroid': centroids, 'Polygon': object_coords, 'Data_values': data_values,
                'Perimeter': perimeters, 'Eccentricity': eccs, 'Orientation': orients}

    features = pd.DataFrame(features, columns=['TimeInds', 'Date', 'Forecast_period',
                                               'Forecast_reference_time', 'Member', 'Threshold', 'ObjectLabel', 'Area',
                                               'Perimeter',
                                               'GridPoints', 'Eccentricity', 'Orientation',
                                               'Mean', 'Std', 'Max', 'Min', 'Centroid', 'Polygon', 'Data_values'])
    return features

In [91]:
# Define a function to compute the convex hull
def compute_convex_hull(points):
    if len(points) > 2:  # A hull can only be formed if there are more than two points
        return MultiPoint(points).convex_hull
    else:
        return MultiPoint(points)

def return_2d_hist(X, Y, xedges = np.arange(90, 180, 1), 
                       yedges = np.arange(0, 55, 1), density=True):
    H, xedges, yedges = np.histogram2d(X, Y, bins=(xedges, yedges), density=density)
    xcenters = (xedges[:-1] + xedges[1:]) / 2
    ycenters = (yedges[:-1] + yedges[1:]) / 2
    print(len(xcenters), len(ycenters), H.shape)
    
    # Histogram does not follow Cartesian convention (see Notes),
    # therefore transpose H for visualization purposes.
    H = H.T
    #X, Y = np.meshgrid(xedges, yedges)
    return xcenters, ycenters, H

In [40]:
cubes = iris.load_cube('/scratch/hadpx/SEA_monitoring/processed_SEA_data/mogreps/features/precip/precip_Features_24h_allMember_20240701.nc')

In [41]:
nmembers, ntime, _, _ = cubes.shape
print(nmembers, ntime)
cubes

36 8


<iris 'Cube' of precipitation_amount / (kg m-2) (realization: 36; time: 8; latitude: 186; longitude: 214)>

In [42]:
thresholds=[5]
threshold_method='geq'
frames = []
for i in range(ntime):
    print_progress_bar(i + 1, ntime)
    for mem in range(nmembers):
        frames.append(grid_features(cubes[mem, i], thresholds=thresholds, time_index=i, member=mem,
                                            threshold_method=threshold_method))

In [43]:
df = pd.concat(frames, ignore_index=True)

In [44]:
sns.scatterplot(data=df, x='Area', y='Max', hue='Member')

<Axes: xlabel='Area', ylabel='Max'>

In [21]:
df.TimeInds.unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [221]:
t = 4
xdf = df.loc[ (df['TimeInds']==t) & (df['Mean']>=10) & (df['Area']>=10) & (df['Area']<200) & (df['Eccentricity']>=0.75)]
xdf['Hull'] = xdf['Polygon'].apply(compute_convex_hull)
xdf.Member.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35])

In [222]:
cube_longitudes = cubes.coord('longitude').points
cube_latitudes = cubes.coord('latitude').points

In [223]:
#xdf['lons'] = [fxx[0] for fxx in xdf.Centroid]
#xdf['lats'] = [fxx[1] for fxx in xdf.Centroid]
longitudes = []
latitudes = []
# Iterate over each row in the 'Polygon' column
for polygon in xdf['Polygon']:
    # Extract the first value (longitude) from each tuple
    longitudes.extend([point[0] for point in polygon])
    latitudes.extend([point[1] for point in polygon])

In [249]:
#H, xedges, yedges = return_2d_hist(longitudes, latitudes, xedges = cube_longitudes, 
#                       yedges = cube_latitudes, density=False)

H, xedges, yedges = np.histogram2d(longitudes, latitudes, bins=(cube_longitudes, cube_latitudes), density=False)
H.shape, len(cube_longitudes), len(cube_latitudes)

((213, 185), 214, 186)

In [253]:
#plt.pcolormesh(H.T)
#plt.colorbar()

In [254]:
ssl_prob = cubes[0, 0].copy()
ssl_prob.rename("Squall probability (%)")
ssl_prob.units = ""

ssl_prob.data[:,:] = 0.
ssl_prob.data[:-1, :-1] = 100.*H.T/36. # Probability in %

In [258]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap

# Create a new colormap that adds white at the start of viridis_r
viridis_r_with_white = LinearSegmentedColormap.from_list(
    'viridis_r_with_white', 
    [(1, 1, 1)] + [mpl.cm.YlGnBu(i) for i in range(mpl.cm.viridis_r.N)]
)

fig = plt.figure(1, figsize=(8, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([90, 140, -10, 20], crs=ccrs.PlateCarree())
# Add geographical features\n",
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', alpha=0.3)
# Add land feature with grey color\n",
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.add_feature(cfeature.COASTLINE)
# Create a colormap that starts with white and transitions to blue
colors = [(1, 1, 1), (0.15, 1, 0.)]  # White to Blue
#cmap = LinearSegmentedColormap.from_list('white_to_blue', colors)
cmap = viridis_r_with_white
#cmap = plt.get_cmap('YlGnBu')
levels=np.arange(0, 30, 5)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
qplt.pcolormesh(ssl_prob, cmap=cmap, norm=norm, alpha=0.95)
plt.gca().set_title(f'Squall probability (%): T+{t*24}h' )
#plt.colorbar()

Text(0.5, 1.0, 'Squall probability (%): T+96h')

In [175]:
xedges[:10], cube_longitudes[:10]

(array([85.078125, 85.359375, 85.640625, 85.921875, 86.203125, 86.484375,
        86.765625, 87.046875, 87.328125, 87.609375]),
 array([85.078125, 85.359375, 85.640625, 85.921875, 86.203125, 86.484375,
        86.765625, 87.046875, 87.328125, 87.609375]))

In [256]:
fig = plt.figure(2, figsize=(8, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([90, 140, -10, 20], crs=ccrs.PlateCarree())
# Add geographical features\n",
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray')
# Add land feature with grey color\n",
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.add_feature(cfeature.COASTLINE)

cmap = plt.get_cmap('YlGnBu')
levels=np.arange(15, 35, 5)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
qplt.pcolormesh(cubes[mem, t], cmap=cmap, norm=norm, alpha=0.5)
#qplt.pcolormesh(cubes[:, t].collapsed('realization', iris.analysis.MEAN), cmap=cmap, norm=norm, alpha=0.4)

for i, row in xdf.iterrows():
    #ydf = xdf.loc[xdf.Member == mem]
    hull = row['Hull']
    if hull.geom_type == 'Polygon':
        x, y = hull.exterior.xy
        plt.fill(x, y, color='pink', alpha=0.5, label='GPM')
        plt.plot(x, y, color='red', linewidth=1)   
    elif hull.geom_type == 'Point':
        plt.scatter(*hull.xy, color='red', s=0.1, label='Point')
        plt.scatter(*zip(*row['Polygon']), s=0.1, color='red', zorder=5)

In [56]:
xdf

,TimeInds,Date,Forecast_period,Forecast_reference_time,Member,Threshold,ObjectLabel,Area,Perimeter,GridPoints,Eccentricity,Orientation,Mean,Std,Max,Min,Centroid,Polygon,Data_values
28129,3,2024-09-11,72.0,NaN,0,5,3_0_5_36,3.0,1.000000,3,1.000000,1.570796,26.729218,19.663903,53.795090,7.666128,"(139.078125, -4.21875)","[(138.796875, -4.21875), (139.078125, -4.21875...","[18.72644, 53.79509, 7.6661277]"
28365,3,2024-09-11,72.0,NaN,1,5,3_1_5_43,40.0,30.349242,40,0.968486,0.410103,24.933186,57.105779,377.837738,5.263803,"(99.703125, -2.15625)","[(98.859375, -3.28125), (98.859375, -3.09375),...","[7.9605165, 14.086887, 13.015654, 13.58514, 7...."
28573,3,2024-09-11,72.0,NaN,2,5,3_2_5_8,3.0,1.000000,3,1.000000,1.570796,23.099701,11.670655,37.569885,8.989536,"(119.953125, -8.53125)","[(119.671875, -8.53125), (119.953125, -8.53125...","[8.989536, 37.569885, 22.739687]"
29624,3,2024-09-11,72.0,NaN,6,5,3_6_5_153,76.0,47.420310,76,0.937679,-0.085727,26.822018,34.512069,188.895508,5.066349,"(107.578125, 15.09375)","[(107.578125, 13.59375), (107.015625, 13.78125...","[5.144165, 8.87067, 53.893448, 8.042408, 8.300..."
29636,3,2024-09-11,72.0,NaN,6,5,3_6_5_165,20.0,16.242641,20,0.960253,-0.235146,27.940976,27.147479,99.744728,5.109013,"(106.453125, 17.71875)","[(106.453125, 16.78125), (106.453125, 16.96875...","[28.495876, 16.344868, 6.43153, 5.353215, 6.25..."
29882,3,2024-09-11,72.0,NaN,8,5,3_8_5_12,57.0,34.763456,57,0.887548,-1.173969,20.129859,14.855059,58.845051,5.698244,"(108.703125, -5.90625)","[(108.984375, -6.65625), (109.265625, -6.65625...","[15.55381, 11.69257, 10.022181, 24.933424, 52...."
30680,3,2024-09-11,72.0,NaN,11,5,3_11_5_157,3.0,3.414214,3,0.816497,-0.785398,21.451416,4.032911,26.745907,16.967644,"(99.984375, 11.71875)","[(99.703125, 11.71875), (99.984375, 11.71875),...","[20.640694, 26.745907, 16.967644]"
31052,3,2024-09-11,72.0,NaN,13,5,3_13_5_26,60.0,30.899495,60,0.822352,-1.174124,22.413094,13.740709,56.170792,5.518701,"(106.453125, -5.15625)","[(105.609375, -5.53125), (105.890625, -5.53125...","[16.189468, 5.5187006, 10.684816, 14.638453, 1..."
31488,3,2024-09-11,72.0,NaN,15,5,3_15_5_25,2.0,0.000000,2,1.000000,0.000000,20.418068,8.989006,29.407074,11.429062,"(120.234375, -5.71875)","[(120.234375, -5.90625), (120.234375, -5.71875)]","[11.429062, 29.407074]"
31759,3,2024-09-11,72.0,NaN,16,5,3_16_5_84,17.0,15.278175,17,0.901828,0.845571,24.503732,35.072754,157.718842,5.030157,"(114.609375, 2.53125)","[(113.765625, 1.96875), (114.046875, 1.96875),...","[9.102145, 5.030157, 6.4072785, 8.6419525, 19...."
